<a href="https://colab.research.google.com/github/oliverrmaa/data-wrangling-springboard/blob/main/solutions/data_cleaning_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

In [ ]:
# Imports:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Google Drive mount authentication
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone instructor's git repo over to access data 
!git clone https://github.com/oliverrmaa/data-wrangling-springboard.git

# 1. Fetch Data & Basic Exploration

Here we will grab data from an Excel File located in our github repo. We will 
use Pandas to extract from the file and also do some brief basic summary level
explorations of the data

In [ ]:
PATH = "/content/data-wrangling-springboard/data/movie_ratings.xlsx"
excel_file = pd.ExcelFile(PATH)

df_list = [excel_file.parse(sheet_name) for sheet_name in excel_file.sheet_names]

In [ ]:
df_movies = df_list[0]
df_rating = df_list[1]

## 1.1 Ratings Dataset

Lets first explore the ratings dataset. We can also assess whether the ratings 
are discrete are continuous. That would be important information to know from
a modeling perspective as a Data Scientist.

In [ ]:
df_rating.head()

In [ ]:
df_rating.info()

In [ ]:
df_rating.describe()

In [ ]:
df_rating["rating"].unique()

In [ ]:
sns.histplot(data=df_rating, x="rating", kde=True)

# 1.2 Movies Dataset

In [ ]:
df_movies.head()

In [ ]:
df_movies.info()

In [ ]:
df_movies.describe()

# 2. Removing Duplicates

First, we want to be aware which columns (or combination of column)
uniquely identify a row. This is also called a surrogate key in database terms. It will make our cleaning much easier if we can identify these things, especially when it comes to duplicates. Afterwards, we can remove these duplicates.

In [ ]:
df_rating[df_rating.duplicated(['userId', 'timestamp'])]

In [ ]:
df_rating[(df_rating.userId == 1) & (df_rating.timestamp == 964981179)]

In [ ]:
df_rating[df_rating.duplicated(['userId', 'timestamp', 'movieId'])]

It seems there are no duplicates in the ratings dataset, data is unique
for each `timestamp`, `userId`, and `movieId`, or in other words, each row
is a unique rating of a movie at a timestamp from a certain user. 

Lets now explore the movies dataset.

In [ ]:
df_movies[df_movies.duplicated(['movieId'])]

In [ ]:
df_movies[df_movies.movieId == 261]

In [ ]:
df_movies = df_movies[df_movies.movieId != 261]

In [ ]:
df_movies[df_movies.movieId == 261]

 For the movies dataset, there was one duplicate entry. "Star Wars" and "Little Women" are definitely not the same movie. We therefore have to drop all 
movieIds with 261 because we cannot recover whether the rating will be for
"Star Wars" or "Little Women" after joining the two datasets.

# 3. Remove Nulls

Now we can assess each column in each dataset for nulls. Let's write a function
this time to iterate through the columns to see how much nulls there are.

In [ ]:
for column in df_rating.columns:
  null_count = len(df_rating[df_rating[column].isna()])
  print("{} : {}".format(column, null_count))

In [ ]:
for column in df_movies.columns:
  null_count = len(df_movies[df_movies[column].isna()])
  print("{} : {}".format(column, null_count))

In [ ]:
df_rating = df_rating.dropna(how='any')

In [ ]:
df_rating[df_rating["rating"].isna()]

# 4. Join Tables

In [ ]:
df = df_rating.merge(df_movies, on='movieId', how='left')

In [ ]:
# Could not recover the movieID for two movies due to duplicate issue
# Some rows will be null, lets just drop those
df = df.dropna(how='any')

In [ ]:
df.head()

# 5. Exercises!

We have one wrangled, cleaned, and consolidated dataset now. But there is more potential work to be done!
1. The `title` column actually has the year of the movie which could be a useful
   feature. Can we extract this? 
2. The `genres` are one giant string, can we make this more useful of a column?
   Perhaps change it to a list. 
3. We saw earlier there were outliers in `ratings`. Can we clean this up a bit?
4. The `timestamp` column is in unix time which is not very user friendly. Can 
   we convert it to something more meaningful?

## 5.1 Excercise: Extract year from title column

In [ ]:
# YOUR CODE GOES HERE 

In [ ]:
df["year"] = df['title'].str.split().str[-1].str.replace("(", "").str.replace(")", "")

In [ ]:
df.head()

## 5.2 Excercise: Convert Genres to a List

Bonus Exercise: How would you filter rows based on this column of lists?

In [ ]:
# YOUR CODE GOES HERE

In [ ]:
df['genres'] = df['genres'].str.split("|")

In [ ]:
# Bonus Exercise:
df[df['genres'].apply(lambda x: 'Action' in x)].head()

## 5.3 Excercise: Convert Unix Timestamp to Human-Readable Timestamp

Hint: Use the pandas method pd.to_datetime()

In [ ]:
# YOUR CODE GOES HERE

In [ ]:
# This was a trick question, some of the timestamps are actually incorrectly formatted
df["timestamp"].astype("str").str.len().unique()

In [ ]:
# The 11 digit timestamps are somewhere 200 years from now
df[df["timestamp"].astype("str").str.len() == 11]

In [ ]:
# Removing them, we still see we have some timestamp drift
df_correct_timestamps = df[df["timestamp"].astype("str").str.len() != 11]
pd.to_datetime(df_correct_timestamps['timestamp'], unit='s').max()

In [ ]:
# Remove wrong digit length first
df = df[df["timestamp"].astype("str").str.len() != 11]

In [ ]:
# Convert to UTC
df["timestamp"] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

In [ ]:
import datetime
now = datetime.datetime.now()
current_year = now.year

df = df[pd.DatetimeIndex(df["timestamp"]).year <= current_year]

## 5.4 Exercise: Remove Outliers

In [ ]:
# YOUR CODE GOES HERE

In [ ]:
Q1 = df['rating'].quantile(0.25)
Q3 = df['rating'].quantile(0.75)

In [ ]:
df["rating"].unique()

In [ ]:
df = df[df["rating"] < 99]